# Model Predictions

Comparing a range of model-feature combinations

In [2]:


import pandas as pd
import glob as glob
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble
import xgboost as xgb

import pipeline_helper
import warnings
warnings.filterwarnings('ignore')

In [3]:

y_columns_list = [['arousal'],['valence']]
models = []
model_names = []
# specify the model

models.append(ensemble.RandomForestRegressor())
model_names.append('random_forest_default')

models.append(svm.SVR(kernel='rbf'))
model_names.append('SVR_RBF')

model = linear_model.LinearRegression()
model_name = 'LinearReg'

models.append(xgb.XGBRegressor())
model_names.append('xgboost')

# Scenario 1

In [10]:
fizz_columns_of_interest = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets', 'trapezius_EMG_Raw',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Raw', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    # 'sub_num',
    # 'vid_num',
]

fizz_columns_of_interest_no_emg = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    # 'sub_num',
    # 'vid_num',
]

fizz_columns_of_interest_no_emg_sub_vid = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    'sub_num',
    'vid_num',
]

fizz_columns_of_interest_sub_vid = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets', 'trapezius_EMG_Raw',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Raw', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    'sub_num',
    'vid_num',
]
zipped_xs = list(zip(['fizz','fizz-emg','fizz-emg_sub_vid','fizz_sub_vid'],[
     fizz_columns_of_interest,
     fizz_columns_of_interest_no_emg,
     fizz_columns_of_interest_no_emg_sub_vid,
     fizz_columns_of_interest_sub_vid]))


scenario = 1

# where to save the results:
# TO DO: add which Features in the save_file_name to test features....
for y_columns in y_columns_list:
    for file_type in ['1hz_zscored','1hz']:
        for model_name, model in list(zip(model_names,models)):
            print(model_name)
            fold_arr,x_names, rmses =[],[],[]
            for x_name,x_cols in zipped_xs: 
                save_file_name = f'/work/abslab/emognition_2023_challenge/results_xgboost/scenario_1/scenario_{scenario}{model_name}_{y_columns[0]}_{x_name}'

                subjects = np.loadtxt(f'/work/abslab/emognition_2023_challenge/data/scenario_1/train/sub_list.txt')  # Subject numbers
                video_numbers = np.loadtxt(f'/work/abslab/emognition_2023_challenge/data/scenario_1/train/video_list.txt')  # Video numbers


                x_folder = f"/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/train/physiology/cv/"
                y_folder = f"/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/train/annotations/cv/"


                # x_files, y_files = get_file_paths(subjects, video_numbers, x_folder, y_folder)
                fold_data = pipeline_helper.k_fold_cross_validation_time(subjects, video_numbers,
                                                                         x_folder, y_folder, file_type)
                rmse_list,_,_ = pipeline_helper.TrainModel(model,fold_data,x_cols,y_columns,save_file_name, verbose=False)
                fold_arr.append(fold_num),x_names.append(x_name),rmses.append(np.mean(rmse_list))

            pd.DataFrame({'folds':fold_arr,'x_col_type':x_names,
                      'rmses':rmses}).to_csv(f'/work/abslab/emognition_2023_challenge/results_xgboost/scenario_{scenario}_{model_name}_across_folds_{y_columns[0]}_{file_type}.csv')

xgboost
xgboost
xgboost
xgboost


# scenario 2

In [6]:
fizz_columns_of_interest_no_emg_sub_vid = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    #'sub_num',
    'vid_num'
]

fizz_columns_of_interest_sub_vid = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets', 'trapezius_EMG_Raw',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Raw', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    #'sub_num',
    'vid_num',
]

zipped_xs = list(zip(['fizz-emg_vid','fizz_vid'],[
    # fizz_columns_of_interest,
    #  fizz_columns_of_interest_no_emg,
     fizz_columns_of_interest_no_emg_sub_vid,
     fizz_columns_of_interest_sub_vid]))




# specify the data info
scenario = 2

# where to save the results:
# TO DO: add which Features in the save_file_name to test features....
for y_columns in y_columns_list:
    for file_type in ['1hz','1hz_zscored']:
        for model_name, model in list(zip(model_names,models)):
            print(model_name)
            fold_arr,x_names, rmses =[],[],[]
            for fold_num in range(0,5):
                print(fold_num)
                for x_name,x_cols in zipped_xs: 
                    save_file_name = f'/work/abslab/emognition_2023_challenge/results_xgboost/scenario_{scenario}/fold_{fold_num}/{model_name}_{y_columns[0]}_{x_name}_{file_type}'

                    subjects = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_sub_list.txt')  # Subject numbers
                    video_numbers = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_video_list.txt')  # Video numbers
                    k = 5  # Number of folds

                    x_folder = f"/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold_num}/train/physiology"
                    y_folder = f"/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold_num}/train/annotations"


                    # x_files, y_files = get_file_paths(subjects, video_numbers, x_folder, y_folder)
                    fold_data = pipeline_helper.k_fold_cross_validation(subjects,
                                                                              video_numbers,
                                                                              k, 
                                                                              x_folder,
                                                                              y_folder, 
                                                                              file_type)
                    rmse_list,_,_ = pipeline_helper.TrainModel(model,fold_data,x_cols,y_columns,save_file_name,
                                                                              verbose=False)
                    fold_arr.append(fold_num),x_names.append(x_name),rmses.append(np.mean(rmse_list))

            pd.DataFrame({'folds':fold_arr,'x_col_type':x_names,
                          'rmses':rmses}).to_csv(f'/work/abslab/emognition_2023_challenge/results_xgboost/scenario_{scenario}_{model_name}_across_folds_{y_columns[0]}_{file_type}_sub-vid_specific.csv')  
            
            
            
            

xgboost
0
1
2
3
4
xgboost
0
1
2
3
4
xgboost
0
1
2
3
4
xgboost
0
1
2
3
4


## lagged features

In [ ]:
feat_columns = [
    'vid_num', 'PPG_Peak_Height',
       'ECG_HR-PPG_HR',
       'PPG_Rate_lagged_5', 'PPG_Rate_future_5',
       'PPG_Rate_rolling_mean_10', 'PPG_Rate_rolling_std_10',
       'PPG_Peaks_lagged_5', 'PPG_Peaks_future_5',
       'PPG_Peaks_rolling_mean_10', 'PPG_Peaks_rolling_std_10',
       'ECG_Rate_lagged_5', 'ECG_Rate_future_5',
       'ECG_Rate_rolling_mean_10', 'ECG_Rate_rolling_std_10',
       'EDA_Clean_lagged_5', 'EDA_Clean_future_5',
       'EDA_Clean_rolling_mean_10', 'EDA_Clean_rolling_std_10',
       'EDA_Tonic_lagged_5', 'EDA_Tonic_future_5',
       'EDA_Tonic_rolling_mean_10', 'EDA_Tonic_rolling_std_10',
       'EDA_Phasic_lagged_5', 'EDA_Phasic_future_5',
       'EDA_Phasic_rolling_mean_10', 'EDA_Phasic_rolling_std_10',
       'SCR_Peaks_lagged_5', 'SCR_Peaks_future_5',
       'SCR_Peaks_rolling_mean_10', 'SCR_Peaks_rolling_std_10',
       'SCR_Height_lagged_5', 'SCR_Height_future_5',
       'SCR_Height_rolling_mean_10', 'SCR_Height_rolling_std_10',
       'SCR_Amplitude_lagged_5', 'SCR_Amplitude_future_5',
       'SCR_Amplitude_rolling_mean_10', 'SCR_Amplitude_rolling_std_10',
       'RSP_Amplitude_lagged_5', 'RSP_Amplitude_future_5',
       'RSP_Amplitude_rolling_mean_10', 'RSP_Amplitude_rolling_std_10',
       'RSP_Rate_lagged_5', 'RSP_Rate_future_5',
       'RSP_Rate_rolling_mean_10', 'RSP_Rate_rolling_std_10',
       'RSP_RVT_lagged_5', 'RSP_RVT_future_5', 'RSP_RVT_rolling_mean_10',
       'RSP_RVT_rolling_std_10', 'RSP_Symmetry_PeakTrough_lagged_5',
       'RSP_Symmetry_PeakTrough_future_5',
       'RSP_Symmetry_PeakTrough_rolling_mean_10',
       'RSP_Symmetry_PeakTrough_rolling_std_10',
       'RSP_Symmetry_RiseDecay_lagged_5',
       'RSP_Symmetry_RiseDecay_future_5',
       'RSP_Symmetry_RiseDecay_rolling_mean_10',
       'RSP_Symmetry_RiseDecay_rolling_std_10', 'skt_lagged_5',
       'skt_future_5', 'skt_rolling_mean_10', 'skt_rolling_std_10','vid_mean_val', 'vid_mean_arousal']
fizz_columns_of_interest_no_emg_sub_vid_mean = [
    'PPG_Rate',
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    #'sub_num',
    'vid_num','vid_mean_val', 'vid_mean_arousal'
]




zipped_xs = list(zip(['lagged_features+vidmeans','fizz-emg_means'],[feat_columns,fizz_columns_of_interest_no_emg_sub_vid_mean]))

# specify the data info
scenario = 2


# where to save the results:
# TO DO: add which Features in the save_file_name to test features....
for y_columns in y_columns_list:
    for file_type in ['1hzphysio_features_calc']:
        for model_name, model in list(zip(model_names,models)):
            fold_arr,x_names, rmses =[],[],[]
            print(model_name)
            for fold_num in range(0,4):
                print(fold_num)
                for x_name,x_cols in zipped_xs: 
                    save_file_name = f'/work/abslab/emognition_2023_challenge/results_xgboost/scenario_{scenario}/fold_{fold_num}/{model_name}_{y_columns[0]}_{x_name}_zscored'

                    subjects = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_sub_list.txt')  # Subject numbers
                    video_numbers = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_video_list.txt')  # Video numbers
                    k = 5  # Number of folds

                    x_folder = f"/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold_num}/train/physiology"
                    y_folder = f"/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold_num}/train/annotations"


                    # x_files, y_files = get_file_paths(subjects, video_numbers, x_folder, y_folder)
                    fold_data = pipeline_helper.k_fold_cross_validation(subjects, video_numbers, k, x_folder, y_folder, file_type)
                    rmse_list,_,_ = pipeline_helper.TrainModel(model,fold_data,x_cols,y_columns,save_file_name, verbose=False)
                    fold_arr.append(fold_num),x_names.append(x_name),rmses.append(np.mean(rmse_list))

            pd.DataFrame({'folds':fold_arr,'x_col_type':x_names,
                          'rmses':rmses}).to_csv(f'/work/abslab/emognition_2023_challenge/results_xgboost/scenario_{scenario}_{model_name}_across_folds_{y_columns[0]}_{file_type}.csv')
            
            
            
            

# Scenario 3

In [8]:
y_columns_list = [['valence'],['arousal']]

models = []
model_names = []
models.append(xgb.XGBRegressor())
model_names.append('xgboost')

fizz_columns_of_interest_no_emg_sub_vid = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    'sub_num',
]

fizz_columns_of_interest_sub_vid = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets', 'trapezius_EMG_Raw',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Raw', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    'sub_num',
    #'vid_num',
]

zipped_xs = list(zip(['fizz-emg_sub','fizz_sub'],[
    # fizz_columns_of_interest,
    #  fizz_columns_of_interest_no_emg,
     fizz_columns_of_interest_no_emg_sub_vid,
     fizz_columns_of_interest_sub_vid]))


# specify the data info
scenario = 3


# where to save the results:
# TO DO: add which Features in the save_file_name to test features....

for file_type in ['1hz','1hz_zscored']:
    for model_name, model in list(zip(model_names,models)):
        fold_arr,x_names, rmses =[],[],[]
        print(model_name)
        for fold_num in range(0,4):
            print(fold_num)
            for x_name,x_cols in zipped_xs: 
                save_file_name = f'/work/abslab/emognition_2023_challenge/results_xgboost/scenario_{scenario}/fold_{fold_num}/{model_name}_{y_columns[0]}_{x_name}_zscored'

                subjects = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_sub_list.txt')  # Subject numbers
                video_numbers = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_video_list.txt')  # Video numbers
                k = 5  # Number of folds

                x_folder = f"/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold_num}/train/physiology"
                y_folder = f"/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold_num}/train/annotations"


                # x_files, y_files = get_file_paths(subjects, video_numbers, x_folder, y_folder)
                fold_data = pipeline_helper.k_fold_cross_validation_video(subjects, video_numbers, k, x_folder, y_folder, file_type)
                rmse_list,_,_ = pipeline_helper.TrainModel(model,fold_data,x_cols,y_columns,save_file_name, verbose=False)
                fold_arr.append(fold_num),x_names.append(x_name),rmses.append(np.mean(rmse_list))

        pd.DataFrame({'folds':fold_arr,'x_col_type':x_names,
                      'rmses':rmses}).to_csv(f'/work/abslab/emognition_2023_challenge/results_xgboost/scenario_{scenario}_{model_name}_across_folds_{y_columns[0]}_{file_type}.csv')      







xgboost
0
1
2
3
xgboost
0
1
2
3


# scenario 4

In [13]:
y_columns_list = [['valence'],['arousal']]

models = []
model_names = []
models.append(xgb.XGBRegressor())
model_names.append('xgboost')

fizz_columns_of_interest_no_emg_sub_vid = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    'sub_num',
]

fizz_columns_of_interest_sub_vid = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets', 'trapezius_EMG_Raw',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Raw', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    'sub_num',
    #'vid_num',
]

zipped_xs = list(zip(['fizz-emg_sub','fizz_sub'],[
    # fizz_columns_of_interest,
    #  fizz_columns_of_interest_no_emg,
     fizz_columns_of_interest_no_emg_sub_vid,
     fizz_columns_of_interest_sub_vid]))


# specify the data info
scenario = 4


for file_type in ['1hz','1hz_zscored']:
    for model_name, model in list(zip(model_names,models)):
        fold_arr,x_names, rmses =[],[],[]
        print(model_name)
        for fold_num in range(0,1):
            print(fold_num)
            for x_name,x_cols in zipped_xs: 
                save_file_name = f'/work/abslab/emognition_2023_challenge/results_xgboost/scenario_{scenario}/fold_{fold_num}/{model_name}_{y_columns[0]}_{x_name}_zscored'

                subjects = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_sub_list.txt')  # Subject numbers
                video_numbers = np.loadtxt(f'/work/abslab/emognition_2023_challenge/tests/scenario_{scenario}_fold_{fold_num}_video_list.txt')  # Video numbers
                k = 2  # Number of folds

                x_folder = f"/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold_num}/train/physiology"
                y_folder = f"/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold_num}/train/annotations"


                # x_files, y_files = get_file_paths(subjects, video_numbers, x_folder, y_folder)
                fold_data = pipeline_helper.k_fold_cross_validation_video(subjects, video_numbers, k, x_folder, y_folder, file_type)
                rmse_list,_,_ = pipeline_helper.TrainModel(model,fold_data,x_cols,y_columns,save_file_name, verbose=False)
                fold_arr.append(fold_num),x_names.append(x_name),rmses.append(np.mean(rmse_list))

        pd.DataFrame({'folds':fold_arr,'x_col_type':x_names,
                      'rmses':rmses}).to_csv(f'/work/abslab/emognition_2023_challenge/results_xgboost/scenario_{scenario}_{model_name}_across_folds_{y_columns[0]}_{file_type}.csv')      







xgboost
0
xgboost
0
